In [1]:
#Load in Prerequisite
import sys
! {sys.executable} -m pip install --upgrade 'xdmod-data>=1.0.0,<2.0.0' python-dotenv tabulate
#Configure formatting
import sys
def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)
get_ipython()._showtraceback = exception_handler
import sys
def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)
get_ipython()._showtraceback = exception_handler
from IPython.display import display, Markdown
def display_df_md_table(df):
    return display(Markdown(df.replace('\n', '<br/>', regex=True).to_markdown()))
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import xdmod_data.themes
pio.templates.default = "timeseries"

In [2]:
#Set up enviroment and connection
from pathlib import Path
from os.path import expanduser
xdmod_data_env_path = Path(expanduser('~/xdmod-data.env'))
try:
    with open(xdmod_data_env_path):
        pass
except FileNotFoundError:
    with open(xdmod_data_env_path, 'w') as xdmod_data_env_file:
        xdmod_data_env_file.write('XDMOD_API_TOKEN=')
    xdmod_data_env_path.chmod(0o600)
#Check if API key file is present
from dotenv import load_dotenv
load_dotenv(xdmod_data_env_path, override=True)
#Connect to sever 
#!NOTE: Update from test sever later on to generate with latest data and such
from xdmod_data.warehouse import DataWarehouse
dw = DataWarehouse('https://xdmod-dev.ccr.xdmod.org/')

In [66]:
import csv
import base64

output_file = 'data_document.csv'
file_header = ["title", "url", "etc"]  # TODO - Finalize format later

# Create a new CSV file to store data to, and add a header row, defined above
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(file_header)

    # Loop through all possible permutations of data querying (being duration, realm, metric, dimensions, filters, dataset_type, and aggregation)
    with dw:
        # Load the following data once to reduce calls
        durations = dw.get_durations()
        realms = dw.describe_realms()
        aggression_units = dw.get_aggregation_units()

        # TODO - May need to find a better way for dw.describe_metrics, dimensions, and filter values to get a local copy
        # To reduce the number of calls and queries - These calls take an input which presently results in multiple calls being needed
        # Maybe make all necessary calls once, store them in a dictionary, and access the data from the dictionary with a key, instead of needing
        # to pass parameters to function calls

        for dur in durations:
            for realm in realms['label']:
                for metric_index, metric_row in dw.describe_metrics(realm).iterrows():
                    metrics_label = metric_row['label']
                    metrics_descriptions = metric_row['description']
                    for dim_index, dim_row in dw.describe_dimensions(realm).iterrows():
                        dimension_label = dim_row['label']
                        dimension_descriptions = dim_row['description']
                        for dataSetType in ["timeseries", "aggregate"]:  # These values are hard coded -> may cause error if new types are later introduced
                            for aggUnit in aggression_units:
#                                 #NOTE !!!! - have special case in filters, problem with if dimension is none - not sure what to do in that case
#                                 for filter_index, filter_row in dw.get_filter_values(realm, (dimension_label if dimension_label != "None" else "Resource")).iterrows():
#                                     filter_label = filter_row['label']

                                # dur - permutations of duration field
                                # realm - permutations of realms
                                # metrics_label - the value for the metric
                                # metrics_descriptions - metric description string
                                # dimension_label
                                # dimension_descriptions
                                # filter_label
                                # dataSetType
                                # aggUnit

                                # metric_index, dim_index (The ID fields of the tables), filter_index - might be used for URL generation???

                                # TODO - figure out what all gets added to the description field and where it comes from
                                Description = "Description: " + metrics_descriptions + "\n" + dimension_descriptions + "\n"
                                writer.writerow([dur, realm, metrics_label, dimension_label, dataSetType, aggUnit, Description])

                              # Create URL
                                config = """
                                {
                                    "featured": false,
                                    "trend_line": false,
                                    "x_axis": {},
                                    "y_axis": {},
                                    "legend": {},
                                    "defaultDatasetConfig": {},
                                    "swap_xy": false,
                                    "share_y_axis": false,
                                    "hide_tooltip": false,
                                    "show_remainder": false,
                                    "timeseries": true,
                                    "title": "untitled query 1",
                                    "legend_type": "bottom_center",
                                    "font_size": 3,
                                    "show_filters": true,
                                    "show_warnings": true,
                                    "data_series": {
                                        "data": [
                                            {
                                                "group_by": "allocation",
                                                "color": "auto",
                                                "log_scale": false,
                                                "std_err": false,
                                                "value_labels": false,
                                                "display_type": "line",
                                                "combine_type": "side",
                                                "sort_type": "value_desc",
                                                "ignore_global": false,
                                                "long_legend": true,
                                                "x_axis": false,
                                                "has_std_err": false,
                                                "trend_line": false,
                                                "line_type": "Solid",
                                                "line_width": 2,
                                                "shadow": false,
                                                "filters": {
                                                    "data": [],
                                                    "total": 0
                                                },
                                                "z_index": 0,
                                                "visibility": null,
                                                "enabled": true,
                                                "metric": "allocated_ace",
                                                "realm": "Allocations",
                                                "category": "Allocations",
                                                "id": -7898317815549915
                                            },
                                            {
                                                "group_by": "allocation",
                                                "color": "auto",
                                                "log_scale": false,
                                                "std_err": false,
                                                "value_labels": false,
                                                "display_type": "line",
                                                "combine_type": "side",
                                                "sort_type": "value_desc",
                                                "ignore_global": false,
                                                "long_legend": true,
                                                "x_axis": false,
                                                "has_std_err": false,
                                                "trend_line": false,
                                                "line_type": "Solid",
                                                "line_width": 2,
                                                "shadow": false,
                                                "filters": {
                                                    "data": [],
                                                    "total": 0
                                                },
                                                "z_index": 1,
                                                "visibility": null,
                                                "enabled": true,
                                                "metric": "allocated_raw_su",
                                                "realm": "Allocations",
                                                "category": "Allocations",
                                                "id": -2730650178340023
                                            },
                                            {
                                                "group_by": "fieldofscience",
                                                "color": "auto",
                                                "log_scale": false,
                                                "std_err": false,
                                                "value_labels": false,
                                                "display_type": "line",
                                                "combine_type": "side",
                                                "sort_type": "value_desc",
                                                "ignore_global": false,
                                                "long_legend": true,
                                                "x_axis": false,
                                                "has_std_err": true,
                                                "trend_line": false,
                                                "line_type": "Solid",
                                                "line_width": 2,
                                                "shadow": false,
                                                "filters": {
                                                    "data": [],
                                                    "total": 0
                                                },
                                                "z_index": 2,
                                                "visibility": null,
                                                "enabled": true,
                                                "metric": "avg_ace",
                                                "realm": "Jobs",
                                                "category": "Jobs",
                                                "id": -5903536683929947
                                            }
                                        ],
                                        "total": 3
                                    },
                                    "aggregation_unit": "Auto",
                                    "global_filters": {
                                        "data": [],
                                        "total": 0
                                    },
                                    "timeframe_label": "Previous month",
                                    "start_date": "2024-04-01",
                                    "end_date": "2024-04-30",
                                    "start": 0,
                                    "limit": 10
                                }
                                """
                            
                                # Encode to Base64
                                config_encoded = base64.b64encode(config.encode('utf-8'))

                                 
                                URL = "https://xdmod-dev.ccr.xdmod.org/#main_tab_panel:metric_explorer?config="+config"=""
                                
                                # Get the data for a specific set of permutations
                                data = dw.get_data(
                                    duration=dur,
                                    realm=realm,
                                    metric=metrics_label,
                                    dimension=dimension_label,
                                    filters={},
                                    dataset_type=dataSetType,
                                    aggregation_unit=aggUnit,
                                )


SyntaxError: invalid syntax (688136009.py, line 54)
